In [1]:
import numpy as np

In [2]:
def simplex(x, A, b, c, j_b):
    m, n = A.shape
    while True:
        A_b, c_b = A[: , j_b], c[j_b]
        B = np.linalg.inv(A_b)
        u = c_b.dot(B)# calc capacity vector
        e = u.dot(A) - c.T# calc estimates

        if np.all(e[j_b == False] >= 0):
            return x, j_b

        j_0 = np.argwhere((e < 0) & (j_b == False)).min()

        z = B.dot(A[: , j_0])

        if np.all(z[: m] <= 0):
            raise Exception('There is no solution.')

        theta = np.array([x[j] / z[i]
            if z[i] > 0
            else np.infty
            for i, j in zip(range(m), np.argwhere(j_b))
        ])

        theta_0, s = theta.min(), theta.argmin()
        j_s = np.argwhere(j_b).ravel()[s]

        # update plan
        mask = (j_b == False)
        mask[j_0] = False
        x[mask] = 0
        x[j_0] = theta_0
        x[j_b] = x[j_b] - theta_0 * z

        # update basis
        j_b[j_s] = False
        j_b[j_0] = True

In [3]:
def inv_helper(A_inv, new_col, j):
    m, _ = A_inv.shape
    l = A_inv.dot(new_col)
    l_hat = l.copy(); l_hat[j] = -1
    q = (-1/l[j]) * l_hat
    Q = np.identity(m); Q[:,j] = q
    return Q.dot(A_inv)

def simplex_plus(x, A, b, c, j_b):
    m, n = A.shape
    B = None
    j_0 = None
    while True:
        A_b, c_b = A[: , j_b], c[j_b]
        if B is None:
            B = np.linalg.inv(A_b)
        else:
            j_b_idxs = np.argwhere(j_b).ravel()
            i = np.argwhere(j_b_idxs == j_0).ravel()[0]
            B = inv_helper(B, A_b[:, i], i)
        u = c_b.dot(B)# calc capacity vector
        e = u.dot(A) - c.T# calc estimates

        if np.all(e[j_b == False] >= 0):
            return x, j_b

        j_0 = np.argwhere((e < 0) & (j_b == False)).min()

        z = B.dot(A[: , j_0])

        if np.all(z[: m] <= 0):
            raise Exception('There is no solution.')

        theta = np.array([x[j] / z[i]
            if z[i] > 0
            else np.infty
            for i, j in zip(range(m), np.argwhere(j_b))
        ])

        theta_0, s = theta.min(), theta.argmin()
        j_s = np.argwhere(j_b).ravel()[s]

        # update plan
        mask = (j_b == False)
        mask[j_0] = False
        x[mask] = 0
        x[j_0] = theta_0
        x[j_b] = x[j_b] - theta_0 * z

        # update basis
        j_b[j_s] = False
        j_b[j_0] = True

## Variant 0.1

In [4]:
A = np.array([
    [0, 1, 4, 1, 0, -3, 5, 0],
    [1, -1, 0, 1, 0, 0, 1, 0],
    [0, 7, -1, 0, -1, 3, 8, 0],
    [1, 1, 1, 1, 0, 3, -3, 1]
])

c = np.array([-5, -2, 3, -4, -6, 0, -1, -5])
b = np.array([6, 10, -2, 15])
x = np.array([4, 0, 0, 6, 2, 0, 0, 5], dtype = np.float)
j_b = x != 0

x, j_b = simplex_plus(x, A, b, c, j_b)
print(x.round(2))
x, j_b = simplex(x, A, b, c, j_b)
print(x.round(2))

[ 10.     0.     2.2    0.     2.6    0.93   0.     0.  ]
[ 10.     0.     2.2    0.     2.6    0.93   0.     0.  ]


## Variant 0.2

In [5]:
A = np.array([
    [0, 1, 4, 1, 0, -3, 1, 0],
    [1, -1, 0, 1, 0, 0, 0, 0],
    [0, 7, -1, 0, -1, 3, -1, 0],
    [1, 1, 1, 1, 0, 3, -1, 1]
])

c = np.array([-5, -2, 3, -4, -6, 0, 1, -5])
b = np.array([6, 10, -2, 15])
x = np.array([10, 0, 1.5, 0, 0.5, 0, 0, 3.5],
    dtype = np.float)
j_b = np.array([True, False, True, False, True,
    False, False, True
])

try:
    simplex(x, A, b, c, j_b)
except:
    print('No solution')

No solution


## Variant 1

In [6]:
A = np.array([
    [0, 1, 4, 1, 0, -8, 1, 5],
    [0, -1, 0, -1, 0, 0, 0, 0],
    [0, 2, -1, 0, -1, 3, -1, 0],
    [1, 1, 1, 1, 0, 3, 1, 1]
])

c = np.array([-5, 2, 3, -4, -6, 0, 1, -5])
b = np.array([36, -11, 10, 20])
x = np.array([4, 5, 0, 6, 0, 0, 0, 5], dtype = np.float)
j_b = x != 0


x, j_b = simplex(x, A, b, c, j_b)
x.round(2)

array([ 0.  ,  9.5 ,  5.33,  1.5 ,  0.  ,  0.  ,  3.67,  0.  ])

## Variant 2

In [7]:
A = np.array([
    [0, 1, 1, 1, 0, -8, 1, 5],
    [0, -1, 0, -7.5, 0, 0, 0, 2],
    [0, 2, 1, 0, -1, 3, -1.4, 0],
    [1, 1, 1, 1, 0, 3, 1, 1]
], dtype=np.float)

c = np.array([-6, -9, -5, 2, -6, 0, 1, 3], dtype=np.float)
b = np.array([15, -45, 1.8, 19], dtype=np.float)
x = np.array([4, 0, 6, 6, 0, 0, 3, 0], dtype = np.float)
j_b = np.array([True, False, True, True, False, False, True, False])

x, j_b = simplex(x, A, b, c, j_b)
x.round(2)

array([ 0.  ,  0.  ,  0.  ,  7.06,  0.  ,  1.8 ,  2.58,  3.96])

## Variant 3

In [8]:
A = np.array([
    [0, -1, 1, -7.5, 0, 0, 0, 2],
    [0, 2, 1, 0, -1, 3, -1.5, 0],
    [1, -1, 1, -1, 0, 3, 1, 1]
], dtype=np.float)

c = np.array([-6, -9, -5, 2, -6, 0, 1, 3], dtype=np.float)
b = np.array([6, 1.5, 10], dtype=np.float)
x = np.array([4, 0, 6, 0, 4.5, 0, 0, 0], dtype = np.float)
j_b = np.array([True, False, True, False, True, False, False, False])

x, j_b = simplex(x, A, b, c, j_b)
x.round(2)

array([  0.  ,   0.75,   0.  ,   2.68,   0.  ,   0.  ,   0.  ,  13.43])

## Variant 4 Ax!=b

In [9]:
A = np.array([
    [2, -1, 1, -7.5, 0, 0, 0, 2],
    [4, 2, -1, 0, 1, 5, -1, -4],
    [1, -1, 1, -1, 0, 3, 1, 1]
], dtype=np.float)

c = np.array([-6, -9, -5, 2, -6, 0, 1, 3], dtype=np.float)
b = np.array([-2, 14, 10], dtype=np.float)
x = np.array([4, 0, 6, 0, 4, 0, 0, 0], dtype = np.float)
j_b = np.array([True, False, True, False, True, False, False, False])

np.all(A.dot(x) == b)
#simplex(x, A, b, c, j_b).round(2)

False

## Variant 5

In [10]:
A = np.array([
    [-2, -1, 3, -7.5, 0, 0, 0, 2],
    [4, 2, -6, 0, 1, 5, -1, -4],
    [1, -1, 0, -1, 0, 3, 1, 1]
])

c = np.array([-6, 9, -5, 2, -6, 0, 1, 3])
b = np.array([-23.5, -24, 2])
x = np.array([0, 0, 0, 5, 4, 0, 0, 7], dtype = np.float)
j_b = np.array([False, False, False, True, True, False, False, True])

try:
    simplex(x, A, b, c, j_b)
except:
    print('No solution')

No solution


## Variant 6 Ax!=b

In [11]:
A = np.array([
    [-2, -1, 1, -7, 0, 0, 0, 2],
    [4, 2, -1, 0, 1, 5, -1, -5],
    [1, 11, 0, 1, 0, 3, 1, 1]
], dtype = np.float)


c = np.array([6, -9, 5, -2, 6, 0, -1, 3], dtype = np.float)
b = np.array([14, -31, 7], dtype = np.float)
x = np.array([4, 0, 6, 0, 4, 0, 0, 0], dtype = np.float)
j_b = np.array([False, False, False, True, True, False, False, True])


np.all(A.dot(x) == b)
#simplex(x, A, b, c, j_b).round(2)

False

## Variant 7

In [12]:
A = np.array([
    [1, 3, 1],
    [0, 2, 1]
], dtype = np.float)


c = np.array([1, 2, 0], dtype = np.float)
b = np.array([4, 2], dtype = np.float)
x = np.array([7, -1, 0], dtype = np.float)
j_b = np.array([True, True, False])



x, j_b = simplex(x, A, b, c, j_b)
x.round(2)

array([ 7., -1.,  0.])

In [13]:
A = np.array([
    [1, 3, 1],
    [0, 2, 1]
], dtype = np.float)


c = np.array([1, 2, 0], dtype = np.float)
b = np.array([4, 2], dtype = np.float)
x = np.array([7, -1, 0], dtype = np.float)
j_b = np.array([True, True, False])



x, j_b = simplex(x, A, b, c, j_b)
x.round(2)

array([ 7., -1.,  0.])